[Documentation](https://pymongo.readthedocs.io/en/stable/tutorial.html)


In [ ]:
from pymongo import MongoClient


class DB:
    def __init__(self):
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client["10Academy"]

    def list_collections(self):
        # Returns collections inside the database
        return self.db.list_collection_names()

    def check_if_collection_exist(self, collection_name: str):
        if not self.list_collections().__contains__(collection_name):
            raise Exception(f"Collection, {collection_name} not found.")

    def insert_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        collection = self.db[collection_name]
        return collection.insert_one(data)

    def insert_many_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        result = self.db[collection_name].insert_many(data)
        return result.inserted_ids

    def find_all(self, collection_name):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find()

    def find(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find({key: value})
    
    def find_by_id(self, collection_name, _id):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find

    def find_one(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find_one({key: value})


In [ ]:
class DBWithSchema:
    def __init__(self) -> None:
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client["10Academy1"]

        self.employee_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["name", "age", "company"],
                "properties": {
                    "name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "age": {
                        "bsonType": "number",
                        "description": "must be an number and is required"
                    },
                    "company": {
                        "bsonType": "objectId",
                        "description": "must be an objectId and is required"
                    }
                }
            }
        }
        self.company_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["name", "country", "city"],
                "properties": {
                    "name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "city": {
                        "bsonType": "string",
                        "description": "must be an string and is required"
                    },
                    "country": {
                        "bsonType": "string",
                        "description": "must be an string and is required"
                    }
                }
            }
        }
        try:
            self.db.create_collection("employee")
            self.db.create_collection("company")
        except Exception as e:
            print(e)

        self.db.command("collMod", "employee", validator=self.employee_validator)
        self.db.command("collMod", "company", validator=self.company_validator)

    def list_collections(self):
        return self.db.list_collection_names()
    
    def get_validation(self, collection_name: str) -> dict:
        self.check_if_collection_exist(collection_name)
        return self.db.get_collection(collection_name).options()
    
    def check_if_collection_exist(self, collection_name: str):
        if not self.list_collections().__contains__(collection_name):
            raise Exception(f"Collection, {collection_name} not found.")

    def insert_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        collection = self.db[collection_name]
        return collection.insert_one(data)

    def insert_many_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        result = self.db[collection_name].insert_many(data)
        return result.inserted_ids

    def find_all(self, collection_name):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find()

    def find(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find({key: value})
    
    def find_by_id(self, collection_name, _id):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find

    def find_one(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find_one({key: value})
    


#### Initialize DB Class

In [ ]:
db = DB()

#### Get list of collections

In [ ]:
db.list_collections()

#### Insert record to collection

In [ ]:
data = {
    "name": "John"
}
inserted_record = db.insert_to_collection('users', data)
print(inserted_record.inserted_id)


#### Insert more than record to collection

In [ ]:
data = [{
    "name": "Tom"
},
    {
        "name": "Jane",
        "age": 33
}
]
inserted_records = db.insert_many_to_collection('users', data)
print(inserted_records)

#### Find All records from collection 

In [ ]:
import pprint

records = db.find_all('users')
for record in records:
    pprint.pprint(record)

#### Filter record from collection

In [ ]:
results = db.find('users', 'name', 'Jane')
for result in results:
    pprint.pprint(result)

In [ ]:
results = db.find('users', 'name', 'Rix')
for result in results:
    pprint.pprint(result)

#### Find one from collection

In [ ]:
result = db.find_one('users', 'name', 'Jane')
pprint.pprint(result)

#### Initialize DBWithSchemaClass

In [ ]:
db_with_schema = DBWithSchema()

#### View validators

In [ ]:
import pprint
company_validator = db_with_schema.get_validation('company')
pprint.pprint(company_validator)

In [ ]:
employee_validator = db_with_schema.get_validation('employee')
pprint.pprint(employee_validator)

#### Create Company

In [ ]:
company_data = [
  {
    "name": "Mercedes-Benz",
    "city": "Stuttgart",
    "country": "Germany"
  },
  {
    "name": "Chevrolet",
    "city": "Detroit",
    "country": "United States"
  }
]
inserted_records = db_with_schema.insert_many_to_collection('company', company_data)
print(inserted_records)

In [ ]:
employee_data = [
  {
    "name": "Jane",
    "age": 24,
    "company": inserted_records[0]
  },
  {
    "name": "John",
    "age": 26,
    "company": inserted_records[1]
  },
  {
    "name": "Amy",
    "age": 23,
    "company": inserted_records[1]
  },
  {
    "name": "Jack",
    "age": 22,
    "company": inserted_records[0]
  },
]
inserted_employees = db_with_schema.insert_many_to_collection('employee', employee_data)
print(inserted_employees)

#### Select all Employees

In [ ]:
import pprint

employees = db_with_schema.find_all('employee')
for employee in employees:
    pprint.pprint(employee)

#### Look UP

In [ ]:
lookup = [
    {
        "$lookup": {
            "from": "employee",
            "localField": "_id",
            "foreignField": "company",
            "as": "employees"
        }
    }
]
employees_with_company = db_with_schema.db.company.aggregate(lookup)
employees_list = list(employees_with_company)
for employee in employees_list:
    pprint.pprint(employee)